In [16]:
import pandas as pd
import xlwings as xw
from pathlib import Path

In [13]:
def obtener_carpetas():
    directorio = Path('../excel')
    lista_carpetas = [fichero.name for fichero in directorio.iterdir() if fichero.is_dir()]
           
    return lista_carpetas

In [19]:
def obtener_archivos(lista_carpetas, indice):

    archivos = Path('..\\excel', lista_carpetas[indice])
    lista_archivo = [fichero.name for fichero in archivos.iterdir() if fichero.is_file()]

    return lista_archivo

In [17]:
carpetas = obtener_carpetas()
carpetas

['2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 'parque 2013']

In [21]:
archivos = obtener_archivos(carpetas, indice = 0)
archivos

['PLANILLA VENTA FLORES VERTICAL dic 2011 (MONO)2.xlsx',
 'PLANILLA VENTA FLORES VERTICAL nov 2011(MONO)2.xls',
 'PLANILLA VENTA FLORES VERTICAL nov 2011(MONO)2.xlsx',
 'PLANILLA VENTA FLORES VERTICAL oct 2011(MONO)2.xls',
 'PLANILLA VENTA FLORES VERTICAL oct 2011(MONO)2.xlsx',
 'PLANILLA VENTA FLORES VERTICAL sep2011(MONO)2.xls']

In [87]:
filename = archivos[5]
password = "0811"

wb = xw.Book(f'../excel/2011/{filename}', password=password)

In [88]:
for i, e in enumerate(wb.sheet_names):
    print(i, ':', e)

0 : PARA IMPRIMIR
1 : 11-09-2011
2 : 17-09-2011
3 : 18-09-2011
4 : 19-09-2011
5 : 20-09-2011
6 : 21-09-2011
7 : 22-09-2011
8 : 23-09-2011
9 : 24-09-2011
10 : 25-09-2011
11 : 28-09-2011
12 : 29-09-2011
13 : 30-09-2011
14 : especial 26-09-11 a 30-09-11
15 : MES SEPTIEMBRE
16 : DEFAULT
17 : PLANILLA FLORES
18 : GRAFICA


In [34]:
#sheet = wb.sheets[18]
#df_control = sheet.used_range.options(pd.DataFrame, header=False, index=False).value

In [37]:
#df_caja = df_control.iloc[14:35, 16:22]

In [39]:
#df_caja

In [89]:
for i in range(1,14):
    numero_hoja = i
    sheet = wb.sheets[numero_hoja]
    df = sheet.used_range.options(pd.DataFrame, header=False, index=False).value
    df2 =df.copy()
    #df2.head()
    
    df_stock = df2.iloc[1:10, 16:25]
    df_precio = df2.iloc[1:3, 5:13]
    df_caja = df2.iloc[14:35, 16:22]

    if numero_hoja <= 1:
        movimientos_stock = pd.DataFrame(columns= ['nombre_hoja', 'descripcion', 'astromelias', 'lisiantus', 'montoneras', 'gladiolo', 'ramito', 'crisantemo', 'rosa', 'clavel'])
        precios = pd.DataFrame(columns= ['nombre_hoja', 'astromelias', 'lisiantus', 'montoneras', 'gladiolo', 'ramito', 'crisantemo', 'rosa', 'clavel'])
        flujo_caja = pd.DataFrame(columns= ['nombre_hoja', '$ SALDO INICIAL', '$ RETIROS', '$ GASTOS', '$ Vtas TOTAL', '$ SALDO', '$ SALDO REAL'])
    
    
    dicc_precios = { 'nombre_hoja' : wb.sheet_names[numero_hoja]}
    df_precio.iloc[0] = df_precio.iloc[0].apply(lambda x: x.lower())
    for i in range(len(df_precio.columns)):
        #print(df_precio.iloc[0,i], ':', df_precio.iloc[1,i])
        dicc_precios[df_precio.iloc[0,i]] = df_precio.iloc[1,i]
    precios = pd.concat([precios, pd.DataFrame(dicc_precios, index= [0])], ignore_index= True)
    
    
    dicc_stock = { 'nombre_hoja' : wb.sheet_names[numero_hoja]}
    df_stock.iat[0,0] = 'descripcion'
    df_stock.iloc[0, 1:] = df_stock.iloc[0, 1:].apply(lambda x: x.lower())
    lista_descripcion = list(df_stock[16])[1:]

    for descripcion in lista_descripcion:
        indice = df_stock.index[df_stock[16] == descripcion].values[0]-1
        #print(descripcion, ':', indice)
        for i in range(len(df_stock.columns)):
            #pass
            #print(df_stock.iloc[0, i], ':', df_stock.iloc[indice, i])
            dicc_stock[df_stock.iloc[0, i]] = df_stock.iloc[indice, i]
        movimientos_stock = pd.concat([movimientos_stock, pd.DataFrame(dicc_stock, index= [0])], ignore_index= True)
    
    
    dicc_caja = { 'nombre_hoja' : wb.sheet_names[numero_hoja],
                    '$ SALDO INICIAL' : 0, 
                    '$ RETIROS' : 0, 
                    '$ GASTOS' : 0, 
                    '$ Vtas TOTAL' : 0, 
                    '$ SALDO' : 0, 
                    '$ SALDO REAL' : 0
                }
    df_caja.reset_index(drop= True, inplace= True)
    indice_saldo_inicial = df_caja.index[df_caja[16] == '$ SALDO INICIAL'].values[0]
    indice_ventas_total = df_caja.index[df_caja[16] == '$ Vtas TOTAL'].values[0]
    indice_saldo = df_caja.index[df_caja[16] == '$ SALDO'].values[0]
    indice_saldo_real = df_caja.index[df_caja[16] == '$ SALDO REAL'].values[0]
    indice_subtotal = df_caja.index[df_caja[19] == 'SUBTOTAL'].values[0]

    dicc_caja['$ SALDO INICIAL'] = round(df_caja.iloc[indice_saldo_inicial, 1],2)
    dicc_caja['$ RETIROS'] = round(df_caja.iloc[indice_subtotal, 4], 2)
    dicc_caja['$ GASTOS'] = round(df_caja.iloc[indice_subtotal, 5], 2)
    dicc_caja['$ Vtas TOTAL'] = round(df_caja.iloc[indice_ventas_total, 1], 2)
    dicc_caja['$ SALDO'] = round(df_caja.iloc[indice_saldo, 1], 2)
    dicc_caja['$ SALDO REAL'] = round(df_caja.iloc[indice_saldo_real, 1], 2)

    flujo_caja = pd.concat([flujo_caja, pd.DataFrame(dicc_caja, index= [0])], ignore_index= True)

In [90]:
precios.to_csv('../csv/2011/precios sep-2011', index= False)
movimientos_stock.to_csv('../csv/2011/movimientos_stock sep-2011', index= False)
flujo_caja.to_csv('../csv/2011/flujo_caja sep-2011', index= False)

In [92]:
precios

,nombre_hoja,astromelias,lisiantus,montoneras,gladiolo,ramito,crisantemo,rosa,clavel
0,11-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69
1,17-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69
2,18-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69
3,19-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69
4,20-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69
5,21-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69
6,22-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69
7,23-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69
8,24-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69
9,25-09-2011,None,None,3.0,4.0,3.0,3.5,5.0,1.69


In [95]:
movimientos_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nombre_hoja  104 non-null    object
 1   descripcion  104 non-null    object
 2   astromelias  65 non-null     object
 3   lisiantus    65 non-null     object
 4   montoneras   78 non-null     object
 5   gladiolo     91 non-null     object
 6   ramito       68 non-null     object
 7   crisantemo   93 non-null     object
 8   rosa         95 non-null     object
 9   clavel       97 non-null     object
dtypes: object(10)
memory usage: 8.2+ KB


In [91]:
flujo_caja

,nombre_hoja,$ SALDO INICIAL,$ RETIROS,$ GASTOS,$ Vtas TOTAL,$ SALDO,$ SALDO REAL
0,11-09-2011,None,450.0,0.0,435.77,-14.23,48.65
1,17-09-2011,None,0.0,0.0,1879.09,1879.09,None
2,18-09-2011,138.15,3150.0,236.0,3323.82,75.97,241.25
3,19-09-2011,241.25,700.0,0.0,575.32,116.57,168.0
4,20-09-2011,168.0,200.0,0.0,264.74,232.74,230.75
5,21-09-2011,230.75,350.0,0.0,250.5,131.25,140.0
6,22-09-2011,140.0,550.0,0.0,484.84,74.84,77.75
7,23-09-2011,77.75,0.0,0.0,202.37,280.12,279.6
8,24-09-2011,279.6,800.0,0.0,740.17,219.77,219.75
9,25-09-2011,219.75,3890.0,392.25,3832.17,-230.33,66.3
